In [2]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import json
import os
import random
import glob
#from detectron_pro import detectron_mask_img,detectron_mask_img_composite
import shutil
import cv2

In [3]:

# to get location that stickimg will sticked on jpg_dir center or random
def center_location(jpg_dir,stickimg_dir):
    im = np.array(Image.open(jpg_dir), dtype=np.uint8)
    x_center,y_center = im.shape[1]/2,im.shape[0]/2
    im_stick = np.array(Image.open(stickimg_dir), dtype=np.uint8)
    im_stick_shape = im_stick.shape
    bd_box_x,bd_box_y = x_center-(im_stick_shape[1]/2),y_center-(im_stick_shape[0]/2)
    bd_box_length,bd_box_height =im_stick_shape[1],im_stick_shape[0]
    return bd_box_x,bd_box_y,bd_box_length,bd_box_height

def random_location(jpg_dir,stickimg_dir):
    im = np.array(Image.open(jpg_dir), dtype=np.uint8)
    # x boundary
    rangeX = 128
    x_left_bound,x_right_bound = rangeX, im.shape[1]-rangeX
    x_center = random.randrange(x_left_bound, x_right_bound)
    seq = [350,300,250,200]
    y_center = random.choice(seq)
    i_shape1 = [128,64]
    i_shape2 = [128*7/8,64*7/8]
    i_shape3 = [128*3/4,64*3/4]
    i_shape4 = [128*2/4,64*2/4]
    dict = {350:i_shape1,300:i_shape2 ,250:i_shape3,200:i_shape4}
    image_shape = dict[y_center]
    bd_box_x,bd_box_y = x_center-(image_shape[1]/2),y_center-(image_shape[0]/2)
    bd_box_length,bd_box_height =image_shape[1],image_shape[0]
    return bd_box_x,bd_box_y,bd_box_length,bd_box_height

In [4]:
#it can generate json file defined by your street_jpg and people_jpg 

def create_json_file(jpg_dir,street_json,stickimg_dir,results_dir,function='center'):
    if function == 'center':
        bd_box_x,bd_box_y,bd_box_length,bd_box_height = center_location(jpg_dir,stickimg_dir)
    elif function == 'random':
        bd_box_x,bd_box_y,bd_box_length,bd_box_height = random_location(jpg_dir,stickimg_dir)
        
    input_file = open (street_json)
    json_array = json.load(input_file)
    
    data = []
    data.append({
    'end':0,
    'hide':0,
    'id':0,
    'init':0,
    'lbl':"person",
    'lock':0,
    'occl':0,
    'pos':[
    bd_box_x,
    bd_box_y,
    bd_box_length,
    bd_box_height],    
    'posv':[
    0,
    0,
    0,
    0],
    'str':0
                })
    
    if json_array != []:
        for item in json_array:
            data.append(item)
            
    with open(results_dir, 'w') as outfile:
        json.dump(data, outfile)

# Start create

In [5]:
# Image read dir
street_dir = '/root/notebooks/0858611-2/final_project/caltech_pedestrian_extractor/video_extractor/*'

people_dir = '/root/notebooks/0858611-2/final_project/caltech_pedestrian_extractor/js_on_image/people_img/Market-1501-v15.09.15'

# Image save dir
save_dir = '/root/notebooks/0858611-2/final_project/caltech_pedestrian_extractor/0603_result'

num_imgs = 10

In [ ]:
# Check dir folder exit
# If not, create one
if os.path.exists(save_dir) == False:
    os.makedirs(save_dir)

for s in ['people', 'mask', 'street', 'street_json','json']:
    if os.path.exists(os.path.join(save_dir, s)) == False:
        os.makedirs(os.path.join(save_dir, s))

In [ ]:
street_imgs = glob.glob(street_dir+'/**/*.jpg', recursive=True)

#street_imgs = random.shuffle(random.sample(street_imgs, 5000))
street_imgs = random.sample(street_imgs, num_imgs)

random.shuffle(street_imgs)

In [ ]:
people_imgs = glob.glob(people_dir+'/bounding_box_train/*.jpg', recursive=True)

people_imgs = random.sample(people_imgs, num_imgs)

random.shuffle(people_imgs)

In [ ]:
for i in range(num_imgs):
    
    if (i%100==0):
        print("Process (",i,"/",num_imgs,")  ","{:.2f}".format(100*i/num_imgs)," %")
    
    # create mask and save
    try:
        mask_img = detectron_mask_img(people_imgs[i],(64,128))
        mask_img = Image.fromarray(mask_img)
    except Exception as e:
        print("Skip image :",i)
        continue
        
    mask_img.save(save_dir+'/mask/'+str('{0:06}'.format(i))+'.jpg')
    
    # save street img
    street_img = cv2.imread(street_imgs[i])
    street_img = cv2.resize(street_img,(640,480))
    cv2.imwrite(save_dir+'/street/'+str('{0:06}'.format(i))+'.jpg', street_img)
    
    ################################################################
    img_path = street_imgs[i]
    json_dir = img_path.replace('images', 'annotations')
    json_dir = json_dir.replace('jpg', 'json')
    shutil.copyfile(json_dir, save_dir+'/street_json/'+str('{0:06}'.format(count))+'.json')
    ################################################################
    
    # save poeple img
    people_img = cv2.imread(people_imgs[i])
    people_img = cv2.resize(people_img,(64,128))
    cv2.imwrite(save_dir+'/people/'+str('{0:06}'.format(i))+'.jpg', people_img)
    
    # create json file and save
    create_json_file(save_dir+'/street/'+str('{0:06}'.format(i))+'.jpg',
                     save_dir+'/street_json/'+str('{0:06}'.format(i))+'.json',
                     save_dir+'/people/'+str('{0:06}'.format(i))+'.jpg',
                     save_dir+'/json/'+str('{0:06}'.format(i))+'.json',
                     function="random")

In [ ]:
import json

json_path = '/root/notebooks/0858611-2/final_project/caltech_pedestrian_extractor/video_extractor/set00/V009/annotations/I00018.json'

input_file = open (json_path)
json_array = json.load(input_file)

In [ ]:
json_array
print(type(json_array[0]))